#Task 1: Preparing the Data


##Step 1: Read the data


In [ ]:
import pandas as pd
import pandas as pd
import datetime as dt
pd.set_option("display.max_columns", None)

In [ ]:
df_ = pd.read_csv("flo_data_20k.csv")

In [ ]:
df = df_.copy()

## Step 2: Define the outlier_thresholds and replace_with_thresholds functions required to suppress outliers

NOTE: When calculating cltv, frequency values must be integer. Therefore, round the lower and upper limits with round()

In [ ]:
def outlier_thresholds(dataframe, variable):
  quartile1 = dataframe[variable].quantile(0.01)
  print(quartile1)
  quartile3 = dataframe[variable].quantile(0.99)
  print(quartile3)
  iqr = quartile3 - quartile1 #interquantile_range
  up_limit = quartile3 + 1.5 * iqr
  low_limit = quartile1 - 1.5 * iqr

  rounded_up_limit = round(up_limit)
  rounded_low_limit = round(low_limit)

  return rounded_low_limit, rounded_up_limit

In [ ]:
outlier_thresholds(df, "order_num_total_ever_online")

1.0
20.0


(-28, 48)

In [ ]:
def replace_with_thresholds(dataframe, variable):
  low_limit, up_limit = outlier_thresholds(dataframe, variable)

  dataframe.loc[dataframe[variable] > up_limit, variable] = up_limit
  #dataframe.loc[dataframe[variable] < low_limit, variable] = low_limit

##Step 3: Suppress the "order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline", "customer_value_total_ever_online" variables if they have outliers.

In [ ]:
print(outlier_thresholds(df, "order_num_total_ever_online"))

1.0
20.0
(-28, 48)


In [ ]:
cols = ["order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline", "customer_value_total_ever_online"]

In [ ]:
for col in cols:
  replace_with_thresholds(df, col)

1.0
20.0
1.0
7.0
19.99
1219.9468000000002
39.99
3143.8104000000035


## Step 4: Omnichannel indicates that customers shop both online and offline platforms. Create new variables for the total number of purchases and expenditures of each customer.

In [ ]:
#Total transaction number = order_num_total_ever_online + order_num_total_ever_offline

df["total_order"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]

In [ ]:
#total spend = customer_value_total_ever_offline + customer_value_total_ever_online

df["total_value"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

In [ ]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order,total_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN],5.0,939.37
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.0,2013.55
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]",5.0,585.32
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]",2.0,121.97
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR],2.0,209.98


##Step 5: Examine the variable types. Change the type of variables expressing date to date.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [ ]:
date_cols = [col for col in df.columns if "date" in col]

In [ ]:
for col in date_cols:
  df[col] = pd.to_datetime(df[col].str.strip(), format= "%Y-%m-%d")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

#Task 2: Creating the CLTV Data Structure

##Step 1: Take 2 days after the date of the last purchase in the data set as the analysis date.


In [ ]:
df["last_order_date"].max()

Timestamp('2021-05-30 00:00:00')

In [ ]:
today_date = dt.datetime(2021, 6, 1)

##Step 2: Create a new cltv dataframe containing customer_id, recency_cltv_weekly, T_weekly, frequency and monetary_cltv_avg values. Monetary value will be expressed as the average value per purchase, recency and tenure values will be expressed in weekly basis.

In [ ]:
df["last_order_date"] - df["first_order_date"]

0        119 days
1       1469 days
2        366 days
3         11 days
4        582 days
           ...   
19940    288 days
19941    296 days
19942    621 days
19943    689 days
19944    277 days
Length: 19945, dtype: timedelta64[ns]

In [ ]:
(df["last_order_date"][0] - df["first_order_date"][0]).days

119

In [ ]:
(df["last_order_date"] - df["first_order_date"]).dt.days

In [ ]:
 type((df["last_order_date"][0] - df["first_order_date"][0]).days)

int

In [ ]:
rec = [(df["last_order_date"][i] - df["first_order_date"][i]).days for i in range(df.shape[0])]

In [ ]:
df["recency"] = rec

In [ ]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order,total_value,recency
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN],5.0,939.37,119
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.0,2013.55,1469
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]",5.0,585.32,366
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]",2.0,121.97,11
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR],2.0,209.98,582


In [ ]:
df["T_weekly"] = ((today_date - df["first_order_date"]) / 7).dt.days

In [ ]:
df["recency_weekly"] = df["recency"] / 7

In [ ]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order,total_value,recency,T_weekly,recency_weekly
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN],5.0,939.37,119,30,17.000000
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.0,2013.55,1469,224,209.857143
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]",5.0,585.32,366,78,52.285714
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]",2.0,121.97,11,20,1.571429
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR],2.0,209.98,582,95,83.142857


In [ ]:
df[df["T_weekly"] < df["recency_weekly"]]

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order,total_value,recency,T_weekly,recency_weekly
14,241f0ad0-afb5-11e9-9757-000d3a38a36f,Ios App,Ios App,2018-12-28,2021-05-30,2021-05-30,2019-10-20,23.0,1.0,139.00,4274.29,"[ERKEK, AKTIFSPOR]",24.0,4413.29,884,126,126.285714
208,26fef404-aae2-11e9-a2fc-000d3a38a36f,Android App,Android App,2019-07-11,2021-05-28,2021-05-28,2020-02-01,2.0,1.0,120.48,318.98,[],3.0,439.46,687,98,98.142857
254,71579c58-b0d3-11e9-9757-000d3a38a36f,Android App,Android App,2019-04-12,2021-05-29,2021-05-29,2019-07-04,10.0,2.0,149.97,2043.84,"[AKTIFCOCUK, COCUK, KADIN, AKTIFSPOR]",12.0,2193.81,778,111,111.142857
381,d4fdf864-2852-11ea-b87a-000d3a38a36f,Ios App,Ios App,2019-09-11,2021-05-30,2021-05-30,2020-03-09,3.0,2.0,230.97,735.39,"[KADIN, AKTIFSPOR]",5.0,966.36,627,89,89.571429
397,7b750fe2-5bcb-11ea-bfee-000d3a38a36f,Desktop,Desktop,2020-02-20,2021-05-28,2021-05-28,2020-02-20,1.0,1.0,100.49,183.33,[],2.0,283.82,463,66,66.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19445,81b198da-a2ee-11e9-a2fc-000d3a38a36f,Desktop,Desktop,2019-03-21,2021-05-28,2021-05-28,2021-03-30,2.0,2.0,230.86,181.98,[],4.0,412.84,799,114,114.142857
19462,531920b8-2a44-11eb-b280-000d3a38a36f,Mobile,Mobile,2020-11-19,2021-05-30,2021-05-30,2020-11-19,1.0,1.0,269.99,250.73,[AKTIFCOCUK],2.0,520.72,192,27,27.428571
19579,e5d25a68-ad55-11e9-a2fc-000d3a38a36f,Desktop,Offline,2018-06-07,2021-05-28,2018-07-16,2021-05-28,2.0,1.0,329.99,400.00,[],3.0,729.99,1086,155,155.142857
19674,fdc69fba-b063-11e9-9757-000d3a38a36f,Mobile,Offline,2018-12-20,2021-05-28,2018-12-20,2021-05-28,1.0,1.0,89.99,205.00,[],2.0,294.99,890,127,127.142857


In [ ]:
df["T_weekly"].max()

437

In [ ]:
df["recency_weekly"].max()

433.42857142857144

In [ ]:
df.groupby("master_id").agg({"recency_weekly":"sum",
                             "T_weekly": "sum",
                             "total_order": "sum",
                             "total_value":"sum"}).head()

,recency_weekly,T_weekly,total_order,total_value
master_id,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,78.571429,80,5.0,776.07
00034aaa-a838-11e9-a2fc-000d3a38a36f,247.571429,290,3.0,269.47
000be838-85df-11ea-a90b-000d3a38a36f,28.285714,58,4.0,722.69
000c1fe2-a8b7-11ea-8479-000d3a38a36f,47.428571,51,7.0,874.16
000f5e3e-9dde-11ea-80cd-000d3a38a36f,43.285714,46,7.0,1620.33


In [ ]:
cltv_df = df.groupby("master_id").agg({"recency": lambda recency: recency,# Recency
                                       "first_order_date": lambda date: (today_date - date.min()).days,# Tenure - Customer age
                                       "total_order": lambda order_num: order_num,#Frequency
                                       "total_value": lambda totalvalue: totalvalue})#Monetary

In [ ]:
cltv_df.head()

,recency,first_order_date,total_order,total_value
master_id,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,550,560,5.0,776.07
00034aaa-a838-11e9-a2fc-000d3a38a36f,1733,2031,3.0,269.47
000be838-85df-11ea-a90b-000d3a38a36f,198,411,4.0,722.69
000c1fe2-a8b7-11ea-8479-000d3a38a36f,332,359,7.0,874.16
000f5e3e-9dde-11ea-80cd-000d3a38a36f,303,323,7.0,1620.33


In [ ]:
cltv_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19945 entries, 00016786-2f5a-11ea-bb80-000d3a38a36f to fffe4b30-18e0-11ea-9213-000d3a38a36f
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   recency           19945 non-null  int64  
 1   first_order_date  19945 non-null  int64  
 2   total_order       19945 non-null  float64
 3   total_value       19945 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 779.1+ KB


In [ ]:
cltv_df.columns = ["recency", "T", "frequency", "monetary"]

In [ ]:
cltv_df.head()

,recency,T,frequency,monetary
master_id,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,550,560,5.0,776.07
00034aaa-a838-11e9-a2fc-000d3a38a36f,1733,2031,3.0,269.47
000be838-85df-11ea-a90b-000d3a38a36f,198,411,4.0,722.69
000c1fe2-a8b7-11ea-8479-000d3a38a36f,332,359,7.0,874.16
000f5e3e-9dde-11ea-80cd-000d3a38a36f,303,323,7.0,1620.33


In [ ]:
cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"] # Average earnings value per transaction

In [ ]:
cltv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,19945.0,666.844472,522.126125,0.00,353.000000,536.000000,766.00,3034.0
T,19945.0,801.302833,523.396883,5.00,517.000000,651.000000,836.00,3060.0
frequency,19945.0,4.978140,4.120919,2.00,3.000000,4.000000,6.00,57.0
monetary,19945.0,152.099009,73.524091,22.49,103.653333,136.906154,182.45,1401.8


In [ ]:
cltv_df["recency"] = cltv_df["recency"] / 7 # Convert it to weekly
cltv_df["T"] = cltv_df["T"] / 7

In [ ]:
cltv_df.head()

,recency,T,frequency,monetary
master_id,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,78.571429,80.000000,5.0,155.214000
00034aaa-a838-11e9-a2fc-000d3a38a36f,247.571429,290.142857,3.0,89.823333
000be838-85df-11ea-a90b-000d3a38a36f,28.285714,58.714286,4.0,180.672500
000c1fe2-a8b7-11ea-8479-000d3a38a36f,47.428571,51.285714,7.0,124.880000
000f5e3e-9dde-11ea-80cd-000d3a38a36f,43.285714,46.142857,7.0,231.475714


In [ ]:
cltv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,19945.0,95.263496,74.589446,0.000000,50.428571,76.571429,109.428571,433.428571
T,19945.0,114.471833,74.770983,0.714286,73.857143,93.000000,119.428571,437.142857
frequency,19945.0,4.978140,4.120919,2.000000,3.000000,4.000000,6.000000,57.000000
monetary,19945.0,152.099009,73.524091,22.490000,103.653333,136.906154,182.450000,1401.800000


#Task 3: Establishment of BG/NBD, Gamma Gamma Models and Calculation of CLTV

##Step 1: Fit the BG/NBD model.


In [ ]:
!pip install lifetimes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.9 MB/s eta 0:00:00


In [ ]:
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

In [ ]:
# Estimate the expected purchases from customers within 3 months and add it to the cltv dataframe as exp_sales_3_month
bgf = BetaGeoFitter(penalizer_coef = 0.001)
bgf.fit(cltv_df["frequency"],
        cltv_df["recency"],
        cltv_df["T"])

<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.17, b: 0.00, r: 3.66>

In [ ]:
cltv_df["exp_sales_3_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(12,
                                                                                       cltv_df["frequency"],
                                                                                       cltv_df["recency"],
                                                                                       cltv_df["T"])

In [ ]:
cltv_df.head()

,recency,T,frequency,monetary,exp_sales_3_month
master_id,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,78.571429,80.000000,5.0,155.214000,0.665679
00034aaa-a838-11e9-a2fc-000d3a38a36f,247.571429,290.142857,3.0,89.823333,0.218284
000be838-85df-11ea-a90b-000d3a38a36f,28.285714,58.714286,4.0,180.672500,0.681762
000c1fe2-a8b7-11ea-8479-000d3a38a36f,47.428571,51.285714,7.0,124.880000,1.003943
000f5e3e-9dde-11ea-80cd-000d3a38a36f,43.285714,46.142857,7.0,231.475714,1.046154


In [ ]:
# Estimate the expected purchases from customers within 6 months and add it to the cltv dataframe as exp_sales_6_month
cltv_df["exp_sales_6_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(4*6,
                                                                                       cltv_df["frequency"],
                                                                                       cltv_df["recency"],
                                                                                       cltv_df["T"])

In [ ]:
cltv_df.head()

,recency,T,frequency,monetary,exp_sales_3_month,exp_sales_6_month
master_id,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,78.571429,80.000000,5.0,155.214000,0.665679,1.331357
00034aaa-a838-11e9-a2fc-000d3a38a36f,247.571429,290.142857,3.0,89.823333,0.218284,0.436567
000be838-85df-11ea-a90b-000d3a38a36f,28.285714,58.714286,4.0,180.672500,0.681762,1.363524
000c1fe2-a8b7-11ea-8479-000d3a38a36f,47.428571,51.285714,7.0,124.880000,1.003943,2.007885
000f5e3e-9dde-11ea-80cd-000d3a38a36f,43.285714,46.142857,7.0,231.475714,1.046154,2.092309


##Step 2: Fit the Gamma-Gamma model. Estimate the average value that customers will leave and add it to the cltv dataframe as exp_average_value.


In [ ]:
ggf = GammaGammaFitter(penalizer_coef = 0.01)
ggf.fit(cltv_df["frequency"], cltv_df["monetary"])

<lifetimes.GammaGammaFitter: fitted with 19945 subjects, p: 4.15, q: 0.47, v: 4.08>

In [ ]:
cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df["frequency"],#Total number of transactions
                                        cltv_df["monetary"])

In [ ]:
cltv_df.head()

,recency,T,frequency,monetary,exp_sales_3_month,exp_sales_6_month,exp_average_value
master_id,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,78.571429,80.000000,5.0,155.214000,0.665679,1.331357,160.117062
00034aaa-a838-11e9-a2fc-000d3a38a36f,247.571429,290.142857,3.0,89.823333,0.218284,0.436567,95.234467
000be838-85df-11ea-a90b-000d3a38a36f,28.285714,58.714286,4.0,180.672500,0.681762,1.363524,187.680921
000c1fe2-a8b7-11ea-8479-000d3a38a36f,47.428571,51.285714,7.0,124.880000,1.003943,2.007885,127.792755
000f5e3e-9dde-11ea-80cd-000d3a38a36f,43.285714,46.142857,7.0,231.475714,1.046154,2.092309,236.368338


##Step 3: Calculate 6-month CLTV and add it to the dataframe with the name cltv.



* Observe the 20 people with the highest CLTV value.

In [ ]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df["frequency"],
                                   cltv_df["recency"],
                                   cltv_df["T"],
                                   cltv_df["monetary"],
                                   time = 6, #How many months of CLTV account do you want? 6 month old
                                   freq = "W", # Frequency information of T / Is the data we entered monthly, weekly or daily? W -> Weekly
                                   discount_rate = 0.01)#There may be various discounts on the products we sell over time. Let's take this into consideration too.

In [ ]:
cltv.head()

master_id
00016786-2f5a-11ea-bb80-000d3a38a36f    223.665974
00034aaa-a838-11e9-a2fc-000d3a38a36f     43.622741
000be838-85df-11ea-a90b-000d3a38a36f    268.503948
000c1fe2-a8b7-11ea-8479-000d3a38a36f    269.223429
000f5e3e-9dde-11ea-80cd-000d3a38a36f    518.898937
Name: clv, dtype: float64

In [ ]:
type(cltv)

pandas.core.series.Series

In [ ]:
cltv_df = cltv_df.merge(cltv, on="master_id", how="left")

In [ ]:
cltv_df.head()

,recency,T,frequency,monetary,exp_sales_3_month,exp_sales_6_month,exp_average_value,clv
master_id,,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,78.571429,80.000000,5.0,155.214000,0.665679,1.331357,160.117062,223.665974
00034aaa-a838-11e9-a2fc-000d3a38a36f,247.571429,290.142857,3.0,89.823333,0.218284,0.436567,95.234467,43.622741
000be838-85df-11ea-a90b-000d3a38a36f,28.285714,58.714286,4.0,180.672500,0.681762,1.363524,187.680921,268.503948
000c1fe2-a8b7-11ea-8479-000d3a38a36f,47.428571,51.285714,7.0,124.880000,1.003943,2.007885,127.792755,269.223429
000f5e3e-9dde-11ea-80cd-000d3a38a36f,43.285714,46.142857,7.0,231.475714,1.046154,2.092309,236.368338,518.898937


In [ ]:
cltv_df.sort_values(by = "clv", ascending = False).head(20)

,recency,T,frequency,monetary,exp_sales_3_month,exp_sales_6_month,exp_average_value,clv
master_id,,,,,,,,
47a642fe-975b-11eb-8c2a-000d3a38a36f,2.857143,7.857143,4.0,1401.800000,1.094385,2.188769,1449.060468,3327.777040
7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.142857,13.142857,11.0,758.085455,1.970108,3.940216,767.360602,3172.394390
f59053e2-a503-11e9-a2fc-000d3a38a36f,51.714286,101.000000,7.0,1106.467143,0.722238,1.444476,1127.611525,1708.982063
625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.285714,74.571429,16.0,501.873750,1.565309,3.130618,506.166665,1662.613492
a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.714286,67.285714,52.0,166.224615,4.656138,9.312276,166.712253,1628.887381
9ce6e520-89b0-11ea-a6e7-000d3a38a36f,3.428571,34.428571,8.0,601.226250,1.265456,2.530912,611.492616,1623.812684
851de3b4-8f0c-11eb-8cb8-000d3a38a36f,8.285714,9.428571,2.0,862.690000,0.793924,1.587847,923.679965,1538.855906
53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.714286,13.000000,17.0,259.865294,2.780689,5.561378,262.072907,1529.227995
55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.571429,58.714286,31.0,228.530000,3.083779,6.167558,229.606946,1485.819156


#Task 4: Creating Segments Based on CLTV Value

##Step 1: Divide all your customers into 4 groups (segments) according to 6-month CLTV and add the group names to the data set.


In [ ]:
cltv_df["segment"] = pd.qcut(cltv_df["clv"], 4, labels = ["D","C","B","A"])

In [ ]:
cltv_df.head()

,recency,T,frequency,monetary,exp_sales_3_month,exp_sales_6_month,exp_average_value,clv,segment
master_id,,,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,78.571429,80.000000,5.0,155.214000,0.665679,1.331357,160.117062,223.665974,B
00034aaa-a838-11e9-a2fc-000d3a38a36f,247.571429,290.142857,3.0,89.823333,0.218284,0.436567,95.234467,43.622741,D
000be838-85df-11ea-a90b-000d3a38a36f,28.285714,58.714286,4.0,180.672500,0.681762,1.363524,187.680921,268.503948,A
000c1fe2-a8b7-11ea-8479-000d3a38a36f,47.428571,51.285714,7.0,124.880000,1.003943,2.007885,127.792755,269.223429,A
000f5e3e-9dde-11ea-80cd-000d3a38a36f,43.285714,46.142857,7.0,231.475714,1.046154,2.092309,236.368338,518.898937,A


## Step 2: Make brief 6-month action suggestions to the management for 2 groups you choose among 4 groups.

In [ ]:
cltv_df.groupby("segment").agg(["count","mean","sum"])

recency                                T                             \
          count        mean            sum count        mean            sum   
segment                                                                       
D          4987  139.000172  693193.857143  4987  162.183219  808807.714286   
C          4986   92.629534  461850.857143  4986  112.818320  562512.142857   
B          4986   81.988367  408794.000000  4986  100.326514  500228.000000   
A          4986   67.427139  336191.714286  4986   82.549711  411592.857143   

        frequency                    monetary                            \
            count      mean      sum    count        mean           sum   
segment                                                                   
D            4987  3.768799  18795.0     4987   93.151603  4.645470e+05   
C            4986  4.404733  21962.0     4986  125.788537  6.271816e+05   
B            4986  5.092659  25392.0     4986  160.636574  8.009340e+05   
A            4986  6.646611  33140.0     4986  228.831142  1.140952e+06   

        exp_sales_3_month                        exp_sales_6_month            \
                    count      mean          sum             count      mean   
segment                                                                        
D                    4987  0.408896  2039.163866              4987  0.817792   
C                    4986  0.525448  2619.884613              4986  1.050896   
B                    4986  0.601105  2997.109743              4986  1.202210   
A                    4986  0.773027  3854.312991              4986  1.546054   

                     exp_average_value                             clv  \
                 sum             count        mean           sum count   
segment                                                                  
D        4078.327732              4987   98.691085  4.921724e+05  4987   
C        5239.769227              4986  132.250592  6.594015e+05  4986   
B        5994.219487              4986  168.000578  8.376509e+05  4986   
A        7708.625982              4986  238.023995  1.186788e+06  4986   

                                   
               mean           sum  
segment                            
D         80.340476  4.006580e+05  
C        138.311508  6.896212e+05  
B        199.532849  9.948708e+05  
A        362.315501  1.806505e+06

In [ ]:
# Our 6-month sales expectation in segment A is above 1.5.
# Our 6-month sales expectation in the B segment is around 1.
# A campaign can be applied to segments A and B.

# Customers in segment A are younger. These may be in the new customer class.
# We can describe those in B as loyal_customers.